# Video Activity classification

# Task: Classify the actions in a breakfast preparation video dataset using sequence models

Datasaet is present in my google drive

In [1]:
from google.colab import drive, files
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
path="/content/gdrive/My Drive/video_action_classification/nndataex"
!ls '/content/gdrive/My Drive/video_action_classification/nndataex'

breakfast_i3d.7z  groundTruth	  read_datasetBreakfast.py
data		  groundTruth.7z  splits


Importing necessary libraries for data loading

In [2]:
import os  
import torch
import numpy as np
import os.path 
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

I used keras to implement the model

Importing necessary keras layers and functions

In [7]:
from keras.preprocessing import sequence
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional,MaxPooling1D
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


Code given in the readBreakfast.py file which loads the data into torch tensor

In [0]:
def _isArrayLike(obj):
    return hasattr(obj, '__iter__') and hasattr(obj, '__len__')

 
def load_data(split_load, actions_dict, GT_folder, DATA_folder):
    file_ptr = open(split_load, 'r')
    content_all = file_ptr.read().split('\n')[1:-1]
    content_all = [x.strip('./data/groundTruth/') + 't' for x in content_all]
  
    all_tasks = ['tea', 'cereals', 'coffee', 'friedegg', 'juice', 'milk', 'sandwich', 'scrambledegg', 'pancake', 'salat']

    data_breakfast = []
    labels_breakfast = []
    tasks_breakfast = []
    for i in tqdm(range(len(content_all))):
        content=content_all[i]
        curr_task = content.split('_')[-1].split('.')[0]
        tasks_breakfast.append(int( all_tasks.index(curr_task)) )

        file_ptr = open( GT_folder + content, 'r')
        curr_gt = file_ptr.read().split('\n')[:-1]
        label_seq, length_seq = get_label_length_seq(curr_gt)

        loc_curr_data = DATA_folder + os.path.splitext(content)[0] + '.gz'
        curr_data = np.loadtxt(loc_curr_data, dtype='float32')

        label_curr_video = []
        for iik in range(len(curr_gt)):
            label_curr_video.append( actions_dict[curr_gt[iik]] )
  
        data_breakfast.append(torch.tensor(curr_data,  dtype=torch.float64 ) )
        labels_breakfast.append(label_curr_video )
 
    return   data_breakfast, labels_breakfast, tasks_breakfast


def get_label_bounds( data_labels):
    labels_uniq = []
    labels_uniq_loc = []
    for kki in range(0, len(data_labels) ):
        uniq_group, indc_group = get_label_length_seq(data_labels[kki])
        labels_uniq.append(uniq_group)
        labels_uniq_loc.append(indc_group)
    return labels_uniq, labels_uniq_loc

def get_label_length_seq(content):
    label_seq = []
    length_seq = []
    start = 0
    length_seq.append(0)
    for i in range(len(content)):
        if content[i] != content[start]:
            label_seq.append(content[start])
            length_seq.append(i)
            start = i
    label_seq.append(content[start])
    length_seq.append(len(content))

    return label_seq, length_seq


def get_maxpool_lstm_data(cData, indices):
    list_data = []
    for kkl in range(len(indices)-1):
        cur_start = indices[kkl]
        cur_end = indices[kkl+1]
        if cur_end > cur_start:
            list_data.append(torch.max(cData[cur_start:cur_end,:],
                                       0)[0].squeeze(0))
        else:
            list_data.append(torch.max(cData[cur_start:cur_end+1,:],
                                       0)[0].squeeze(0))
    list_data  =  torch.stack(list_data)
    return list_data

def read_mapping_dict(mapping_file):
    file_ptr = open(mapping_file, 'r')
    actions = file_ptr.read().split('\n')[:-1]

    actions_dict=dict()
    for a in actions:
        actions_dict[a.split()[1]] = int(a.split()[0])

    return actions_dict

Due to memory constraints on colab I had to further split the files into 3 files each for a total of 3*4=12 files

In [5]:
split = 'test'
COMP_PATH = path

train_split =  os.path.join(COMP_PATH, 'splits/train.split22.bundle')
test_split  =  os.path.join(COMP_PATH, 'splits/test.split4.bundle')
GT_folder   =  os.path.join(COMP_PATH, 'groundTruth/')
DATA_folder =  os.path.join(COMP_PATH, 'data/')
mapping_loc =  os.path.join(COMP_PATH, 'splits/mapping_bf.txt')


actions_dict = read_mapping_dict(mapping_loc)
if split == 'train':
    data_feat, data_labels, tasks_labels = load_data(train_split, actions_dict, GT_folder, DATA_folder)
else:
    data_feat, data_labels, tasks_labels= load_data( test_split, actions_dict, GT_folder, DATA_folder)
#print(data_feat.shape,data_labels.shape,tasks_labels.shape)
print('total number videos ' +  str(len(data_labels))  )
 

100%|██████████| 576/576 [10:53<00:00,  1.01s/it]

total number videos 576


The split was made such that each file contains 500 video segments 

In [6]:
print(len(data_feat),len(data_labels),len(tasks_labels))

576 576 576


In [0]:
maxv=0
for i in range(len(data_feat)):
  if data_feat[i].shape[0]>maxv:
    maxv=data_feat[i].shape[0]
    
print(maxv)
  

Each video segment is of different length and needs to be padded

In [8]:
data_feat[0].shape

torch.Size([453, 400])

Padding the features tensor with zero vectors and the data labels with 0 (SILENCE) and converting the labels to one hot encodings (of 48 classes)

In [0]:
data_feat=sequence.pad_sequences(data_feat, maxlen=8500,padding='post', truncating='post', value=0.0)
data_labels=sequence.pad_sequences(data_labels, maxlen=8500,padding='post', truncating='post', value=0)
data_labels = [to_categorical(i, 48) for i in data_labels]
data_feat=np.array(data_feat)
data_labels=np.array(data_labels)

The task labels contains a number between 0-9 indicating 10 different tasks i.e. cereals, coffee, fried egg, milk, salat, sandwich, tea, scrambled egg, pancake, juice


In [0]:
tasks_labels=[to_categorical(i,10) for i in tasks_labels]
tasks_labels=np.array(tasks_labels)
tasks_labels=tasks_labels.reshape(-1,1,10)

Splitting the data into train and validation to evaluate model performance
I used a 90-10 split

In [0]:

#X_train, X_val, y_train, y_val = train_test_split(data_feat, tasks_labels, test_size=0.1, random_state=420)
X_train, X_val, y_train, y_val = train_test_split(data_feat, data_labels, test_size=0.1, random_state=420)

The model is described below, It consists of two bidirectional LSTM layers of 400 output values each followed by a Max pooling layer over each LSTM cell which gives an output vector of size 800
Finally a Dense softmax layer follows which does the actual classification into one of the 10 activities

## Task classification
### The first model is for the task classification problem 
### 1) LSTM model for task classification
Here I use a LSTM model for reasons explained in the abstract

In [0]:
model = Sequential()
model.add(Bidirectional(LSTM(400,return_sequences=True),input_shape=(None,400)))
model.add(Bidirectional(LSTM(400,return_sequences=True)))
model.add(MaxPooling1D(pool_size=8500, strides=None, padding='valid', data_format='channels_last'))

model.add(Dense(10, activation='softmax'))
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])


In [0]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_5 (Bidirection (None, None, 800)         2563200   
_________________________________________________________________
bidirectional_6 (Bidirection (None, None, 800)         3843200   
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, None, 800)         0         
_________________________________________________________________
dense_3 (Dense)              (None, None, 10)          8010      
Total params: 6,414,410
Trainable params: 6,414,410
Non-trainable params: 0
_________________________________________________________________


In [0]:
model=load_model(path+"task_classification.h5")

Using Keras callbacks to save only the best model aftereach epoch

Due to memory and time constraints in colab I had to:


*   Further split the input files
*   Train for 3 epochs only (Training rate of Bi-LSTM is incredibly slow)
*   Use a small batch size as colab crashed on increasing it any further




In [0]:
best_model_file = path+"task_classification.h5"
best_model = ModelCheckpoint(best_model_file, monitor='val_loss', verbose=1, save_best_only=True)

print('Training model...')
results = model.fit(X_train, y_train, epochs=3, batch_size=4, validation_data=(X_val, y_val), callbacks=[best_model])

print('Training finished.')


Training model...
Train on 414 samples, validate on 46 samples
Epoch 1/3
340/414 [=======================>......] - ETA: 12:08 - loss: 0.6186 - acc: 0.7324

Model was evaluated later after importing the test bundle and it has an accuracy of 78.47%

In [0]:
model.evaluate(data_feat, tasks_labels) # This is actually the test data I imported later and not the train data

576/576 [==============================] - 190s 330ms/step


[0.5722099526060952, 0.7847222222222222]

## Activity classification
### 2) Simple neural network model for activity classification
Now moving on to the primary task of activity classification

First I try a simple neural network for classifying the feature vectors into one of the 48 classes

In [0]:
model2 = Sequential()
model2.add(Dense(256,activation='relu',input_shape=(None,400)))
model2.add(Dense(128,activation='relu'))
model2.add(Dense(128,activation='relu'))


model2.add(Dense(48, activation='softmax'))
model2.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
model2.summary()

In [0]:
model2=load_model(path+"label_classification.h5")

In [0]:

best_model2_file = path+"label_classification.h5"
best_model2 = ModelCheckpoint(best_model2_file, monitor='val_loss', verbose=1, save_best_only=True)

print('Training model...')
results = model2.fit(X_train, y_train, epochs=20, batch_size=4, validation_data=(X_val, y_val), callbacks=[best_model2])

print('Training finished.')


Training model...
Train on 449 samples, validate on 50 samples
Epoch 1/20
449/449 [==============================] - 8s 18ms/step - loss: 0.5189 - acc: 0.8500 - val_loss: 0.5757 - val_acc: 0.8391

Epoch 00001: val_loss improved from inf to 0.57570, saving model to /content/gdrive/My Drive/video_action_classification/nndataexlabel_classification.h5
Epoch 2/20
449/449 [==============================] - 7s 16ms/step - loss: 0.4691 - acc: 0.8600 - val_loss: 0.5333 - val_acc: 0.8427

Epoch 00002: val_loss improved from 0.57570 to 0.53334, saving model to /content/gdrive/My Drive/video_action_classification/nndataexlabel_classification.h5
Epoch 3/20
449/449 [==============================] - 7s 16ms/step - loss: 0.4464 - acc: 0.8659 - val_loss: 0.5288 - val_acc: 0.8460

Epoch 00003: val_loss improved from 0.53334 to 0.52883, saving model to /content/gdrive/My Drive/video_action_classification/nndataexlabel_classification.h5
Epoch 4/20
449/449 [==============================] - 7s 16ms/step -

The training process took a few seconds for each training bundle and gave a test accuracy of 87.4%

In [0]:
model2.evaluate(data_feat, data_labels) # This is actually the test data I imported later and not the train data

576/576 [==============================] - 8s 13ms/step


[0.43752316137154895, 0.874196492963367]

### 3) LSTM model for activity classification
There is still scope for improvement, let's see if an LSTM model is able to outperform the vanilla neural network

In [13]:
model3 = Sequential()
model3.add(Bidirectional(LSTM(400,return_sequences=True),input_shape=(None,400)))


model3.add(Dense(48, activation='softmax'))
model3.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
model3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection (None, None, 800)         2563200   
_________________________________________________________________
dense_1 (Dense)              (None, None, 48)          38448     
Total params: 2,601,648
Trainable params: 2,601,648
Non-trainable params: 0
_________________________________________________________________


In [0]:
results = model3.fit(X_train, y_train, epochs=20, batch_size=4, validation_data=(X_val, y_val))

Train on 518 samples, validate on 58 samples
Epoch 1/20
392/518 [=====================>........] - ETA: 14:02 - loss: 0.8984 - acc: 0.8066